In [1]:
!git clone https://github.com/rkuo2000/yolov5
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 3390, done.
remote: Total 3390 (delta 0), reused 0 (delta 0), pack-reused 3390
Receiving objects: 100% (3390/3390), 7.02 MiB | 17.66 MiB/s, done.
Resolving deltas: 100% (2273/2273), done.
/content/yolov5


In [2]:
!pip install pycocotools

In [24]:
!pip install split-folders

In [38]:
!pip install -U PyYAML

     |████████████████████████████████| 636 kB 5.1 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install -q kaggle

In [25]:
import os
import shutil
import numpy as np
import tqdm

import splitfolders

from google.colab import files

In [5]:
from utils.google_utils import *
attempt_download('weights/yolov5s.pt')
attempt_download('weights/yolov5m.pt')
attempt_download('weights/yolov5l.pt')
attempt_download('weights/yolov5x.pt')

  0%|          | 0.00/14.5M [00:00<?, ?B/s]

  0%|          | 0.00/41.9M [00:00<?, ?B/s]

  0%|          | 0.00/91.6M [00:00<?, ?B/s]

  0%|          | 0.00/170M [00:00<?, ?B/s]

In [8]:
%cd /content/

/content


In [9]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"scutumserpens","key":"9ca6990ac4025a3d2b4fd2f602910725"}'}

In [13]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [14]:
! cp kaggle.json ~/.kaggle/

In [15]:
! chmod 600 ~/.kaggle/kaggle.json

In [16]:
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              233KB  2021-08-20 08:51:07          10939  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           6696  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           2529  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-07-03 18:37:20           2684  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Jo

In [17]:
! kaggle datasets download kneroma/tacotrashdataset

100% 2.79G/2.79G [00:44<00:00, 42.3MB/s]
100% 2.79G/2.79G [00:44<00:00, 67.2MB/s]


In [ ]:
!unzip tacotrashdataset.zip

In [19]:
from pycocotools.coco import COCO
data_source = COCO(annotation_file='/content/data/annotations.json')

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [20]:
# remapping label id to 0~1
label_transfer = {5: 0, 12: 1}

In [21]:
img_ids = data_source.getImgIds()

catIds = data_source.getCatIds()
categories = data_source.loadCats(catIds)
categories.sort(key=lambda x: x['id'])
classes = {}
coco_labels = {}
coco_labels_inverse = {}
for c in categories:
    coco_labels[len(classes)] = c['id']
    coco_labels_inverse[c['id']] = len(classes)
    classes[c['name']] = len(classes)

class_num = {}

In [22]:
!mkdir -p tmp/labels tmp/images
save_base_path  = 'tmp/labels/'
save_image_path = 'tmp/images/'

In [23]:
for index, img_id in tqdm.tqdm(enumerate(img_ids), desc='change .json file to .txt file'):
    img_info = data_source.loadImgs(img_id)[0]
 
    save_name = img_info['file_name'].replace('/', '_')
    
    file_name = save_name.split('.')[0]
  
    height = img_info['height']
    width = img_info['width']
    
    save_path = save_base_path + file_name + '.txt'
    is_exist = False  
    with open(save_path, mode='w') as fp:
       
        annotation_id = data_source.getAnnIds(img_id)
        boxes = np.zeros((0, 5))
        if len(annotation_id) == 0:
            fp.write('')
            continue
       
        annotations = data_source.loadAnns(annotation_id)
        lines = '' 
    
        for annotation in annotations:
           
            label = coco_labels_inverse[annotation['category_id']]
            if label in label_transfer.keys():
         
                is_exist = True
                box = annotation['bbox']
                if box[2] < 1 or box[3] < 1:
                  
                    continue
                # top_x,top_y,width,height==>cen_x,cen_y,width,height
                box[0] = round((box[0] + box[2] / 2) / width, 6)
                box[1] = round((box[1] + box[3] / 2) / height, 6)
                box[2] = round(box[2] / width, 6)
                box[3] = round(box[3] / height, 6)
                label = label_transfer[label]  
                if label not in class_num.keys():
                    class_num[label] = 0
                class_num[label] += 1
                lines = lines + str(label)  
                for i in box:  
                    lines += ' ' + str(i)
                lines += '\n'  
        fp.writelines(lines)
    if is_exist:
  
        shutil.copy('/content/data/{}'.format(img_info['file_name']), os.path.join(save_image_path, save_name))
    else:
    
        os.remove(save_path)

change .json file to .txt file: 1500it [00:01, 950.59it/s]


In [26]:
splitfolders.ratio('tmp', output="taco", seed=1337, ratio=(.8, 0.1,0.1)) 

Copying files: 708 files [00:01, 398.12 files/s] 


In [35]:

%cd yolov5

data	    hubconf.py	 models       requirements.txt	tutorial.ipynb
detect.py   kaggle.json  __pycache__  test.py		utils
Dockerfile  LICENSE	 README.md    train.py		weights


In [42]:
!python3 train.py --data coco.yaml --cfg yolov5s.yaml --weights '' --batch-size 64


Using torch 1.9.0+cu102 CPU

Namespace(adam=False, batch_size=64, bucket='', cache_images=False, cfg='./models/yolov5s.yaml', data='./data/coco.yaml', device='', epochs=300, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', rect=False, resume=False, save_dir='runs/train/exp2', single_cls=False, sync_bn=False, total_batch_size=64, weights='', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
Hyperparameters {'lr0': 0.01, 'lrf': 0.2, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1, 'box': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0

In [41]:
!python train.py --img 320 --batch 1 --epochs 100 --data /kaggle/input/yolov5-taco-models/taco2.yaml --cfg models/yolov5s.yaml --weights /kaggle/input/yolov5-taco-models/taco2_epoch300_best.pt

Traceback (most recent call last):
  File "train.py", line 481, in <module>
    opt.data, opt.cfg, opt.hyp = check_file(opt.data), check_file(opt.cfg), check_file(opt.hyp)  # check files
  File "/content/yolov5/utils/general.py", line 70, in check_file
    assert len(files), 'File Not Found: %s' % file  # assert file was found
AssertionError: File Not Found: /kaggle/input/yolov5-taco-models/taco2.yaml
